# Import Modules and Dataset

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import random
from sklearn.impute import SimpleImputer
import gc

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
          'axes.titleweight': 'bold',
          'axes.labelweight': 'bold',
         'xtick.labelsize':'xx-large',
          'xtick.major.size': 5.5,
         'ytick.labelsize':'xx-large',
          'ytick.major.size': 5.5,
          'axes.labelpad': 15.0,
          'grid.color': 'gray',
         'axes.edgecolor': 'black'}
pylab.rcParams.update(params)
plt.rcParams["patch.force_edgecolor"] = True

In [2]:
file = '/Users/charisameeker/Documents/Data/Iowa Liquor/Iowa_Liquor_Sales.csv'
df = pd.read_csv(file, parse_dates=['Date'],
        dtype = {'Store Number':np.int16, 'Item Number':np.int32, 'Pack':np.int16,
                'Volume Sold (Liters)':np.float32, 'Bottles Sold':np.int16,
                'Volumes Sold':np.int16})

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Memory Usage

In [3]:
print('Memory Usage (Deep=True) (GB):',
      np.round(df.memory_usage(deep=True).sum() / 10 ** 9, decimals=2),
    '\nMemory Usage (Deep=False) (GB):',
      np.round(df.memory_usage(deep=False).sum() / 10 ** 9, decimals=2))

Memory Usage (Deep=True) (GB): 12.6 
Memory Usage (Deep=False) (GB): 2.09


# Data Cleaning

In [4]:
# Convert columns to lowercase
df.columns = df.columns.str.lower()

### There are as many invoice numbers as rows, proving this column useless. Let's drop it in the cell below.

In [6]:
df.drop(['invoice/item number', 'volume sold (gallons)'], axis=1, inplace=True)

In [7]:
df.rename(columns = {'store number': 'store_number', 'store name': 'store_name', 'zip code': 'zip_code',
                    'store location': 'store_location', 'country number':'county_number',
                    'category name':'category_name', 'vendor number':'vendor_number', 'vendor name':'vendor_name',
                    'item number':'item_number', 'item description':'item_description', 'bottle volume (ml)':
                    'bottle_vol', 'state bottle cost': 'bottle_cost','state bottle retail':'state_bottle_retail',
                    'bottles sold':'bottles_sold','sale (dollars)':'sale','volume sold (liters)':'vol_sold', # in liters
                    'county number': 'county_number'}, inplace=True)

### Place in Chronological Order

In [8]:
df.sort_values('date', inplace=True)
df.reset_index(drop=True, inplace=True)
df.date.min(), df.date.max()

(Timestamp('2012-01-03 00:00:00'), Timestamp('2017-10-31 00:00:00'))

### Make date column the first column

In [9]:
cols = list(df)
cols.insert(0, cols.pop(cols.index('date')))

In [10]:
df = df.reindex(columns = cols)

### Insert year, month, and day columns

In [11]:
year = pd.to_datetime(df.date).dt.year
year = year.astype(np.int16)
df.insert(1, 'year', year, allow_duplicates=True)

month = pd.to_datetime(df.date).dt.month
month = month.astype(np.int8)
df.insert(2, 'month', month, allow_duplicates=True)

day = pd.to_datetime(df.date).dt.day
day = day.astype(np.int8)
df.insert(3, 'day', day, allow_duplicates=True)

## The table of data: df

In [12]:
pd.set_option('display.max_columns', None)
df.head(3)

,date,year,month,day,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_vol,bottle_cost,state_bottle_retail,bottles_sold,sale,vol_sold
0,2012-01-03,2012,1,3,3944,Sam's Club 4973 / Dubuque,4400 ASBURY RD,DUBUQUE,52002,"4400 ASBURY RD\nDUBUQUE 52002\n(42.515282, -90...",31.0,Dubuque,1062200.0,PUERTO RICO & VIRGIN ISLANDS RUM,35.0,"Bacardi U.S.A., Inc.",43128,Bacardi Superior Rum,6,1750,$14.67,$22.00,18,$396.00,31.5
1,2012-01-03,2012,1,3,2649,Hy-Vee #3 / Dubuque,400 LOCUST ST,DUBUQUE,52001,"400 LOCUST ST\nDUBUQUE 52001\n(42.497219, -90....",31.0,Dubuque,1081900.0,MISC. AMERICAN CORDIALS & LIQUEURS,305.0,MHW Ltd,77570,Travis Hasse Apple Pie Liqueur,12,750,$7.94,$11.91,4,$47.64,3.0
2,2012-01-03,2012,1,3,3744,Payless Foods / Dyersville,733 16TH AVE SE,DYERSVILLE,52040,"733 16TH AVE SE\nDYERSVILLE 52040\n(42.470204,...",31.0,Dubuque,1081335.0,RASPBERRY SCHNAPPS,434.0,Luxco-St Louis,82146,Arrow Raspberry Schnapps,12,750,$3.97,$5.95,12,$71.40,9.0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12591077 entries, 0 to 12591076
Data columns (total 25 columns):
 #   Column               Dtype         
---  ------               -----         
 0   date                 datetime64[ns]
 1   year                 int16         
 2   month                int8          
 3   day                  int8          
 4   store_number         int16         
 5   store_name           object        
 6   address              object        
 7   city                 object        
 8   zip_code             object        
 9   store_location       object        
 10  county_number        float64       
 11  county               object        
 12  category             float64       
 13  category_name        object        
 14  vendor_number        float64       
 15  vendor_name          object        
 16  item_number          int32         
 17  item_description     object        
 18  pack                 int16         
 19  bottle_vol         

# Data Cleaning:
## Finding Missing Values and Stripping String Characters from Numeric Columns

In [14]:
def nan_percentage(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum() / df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Missing Percent'])
    missing_data['Missing Percent'] = missing_data['Missing Percent'].apply(lambda x: x * 100)
    print(missing_data[missing_data['Total'] > 0].round({'Missing Percent': 3}))
    
nan_percentage(df)

                     Total  Missing Percent
county_number        79178            0.629
county               79178            0.629
category_name        16086            0.128
category              8020            0.064
zip_code              2420            0.019
address               2376            0.019
store_location        2375            0.019
city                  2375            0.019
sale                    10            0.000
bottle_cost             10            0.000
state_bottle_retail     10            0.000
vendor_number            3            0.000
vendor_name              1            0.000


## Let's address the missing values in descending order.

## 1. County and County Number

In [15]:
df.columns

Index(['date', 'year', 'month', 'day', 'store_number', 'store_name', 'address',
       'city', 'zip_code', 'store_location', 'county_number', 'county',
       'category', 'category_name', 'vendor_number', 'vendor_name',
       'item_number', 'item_description', 'pack', 'bottle_vol', 'bottle_cost',
       'state_bottle_retail', 'bottles_sold', 'sale', 'vol_sold'],
      dtype='object')

In [16]:
df.county_number.isna().sum()

79178

In [17]:
df.query('county_number.isnull() & county.isnull()', engine='python').shape[0]

79178

In [18]:
df.county_number = df.county_number.fillna(100)
df.county = df.county.fillna(100)
df.county_number = df.county_number.astype(np.int8)

### All rows missing values for the 'county' column also lack rows for the 'county_number' column; they are lost and cannot be recovered with this dataset.

## 2. Category and Category Name

In [19]:
pd.set_option('display.max_columns', None)
df.query('category_name.isnull()',engine='python').shape[0]

16086

In [20]:
df.category_name.dtype, df.category_name.isna().sum()

(dtype('O'), 16086)

### Here's a row with a NaN value as a string:

In [21]:
df.category_name.isna().sum(), df.category_name[5457]

(16086, nan)

### We'll fill in about half of those NaN values (~8,000) with a dictionary

In [22]:
df.category_name = df.category_name.astype(str)

In [23]:
list_num = df.category.value_counts().index.to_list()
list_num[0:4]

[1031080.0, 1012100.0, 1011200.0, 1031200.0]

In [24]:
list_name = df.category_name.value_counts().index.to_list()
list_name[0:4]

['VODKA 80 PROOF',
 'CANADIAN WHISKIES',
 'STRAIGHT BOURBON WHISKIES',
 'SPICED RUM']

In [25]:
def sample_from_dict(d, sample=4):
    keys = random.sample(list(d), sample)
    values = [d[k] for k in keys]
    return dict(zip(keys, values))

In [26]:
merged_dict = dict(zip(list_num, list_name))
sample_from_dict(merged_dict)

{1082000.0: 'SINGLE MALT SCOTCH',
 1062400.0: 'American Flavored Vodka',
 1062050.0: 'Cocktails / RTD',
 1082015.0: 'Holiday VAP'}

In [27]:
def fillNan(row,axis=1):
    if row.category_name == 'nan':
        return merged_dict.get(row.category)
    else:
        return row.category_name

df.category_name = df.apply(fillNan,axis=1)

# [fillNan(i) for i in df.category_name if row.category_name == 'nan']

In [28]:
df.category_name[5457]

'American Schnapps'

In [29]:
# The numbers are not overwritten by strings in category
df.category.head(3)

0    1062200.0
1    1081900.0
2    1081335.0
Name: category, dtype: float64

In [30]:
print('category_name NaN count:', df.category_name.isna().sum(),
      '\ncategory NaN count:', df.category.isna().sum())

category_name NaN count: 8020 
category NaN count: 8020


In [31]:
df.category = df.category.fillna(100) # 89 distinct strings

### The rows of the category and category_name columns wherein both have NaN values (8,020) are lost for good.

## 3. Zip Code, Address, City

In [32]:
df.zip_code.isna().sum()

2420

In [33]:
df.query('zip_code.isna()', engine='python').isna().sum()

date                      0
year                      0
month                     0
day                       0
store_number              0
store_name                0
address                2375
city                   2375
zip_code               2420
store_location         2375
county_number             0
county                    0
category                  0
category_name             4
vendor_number             0
vendor_name               0
item_number               0
item_description          0
pack                      0
bottle_vol                0
bottle_cost               0
state_bottle_retail       0
bottles_sold              0
sale                      0
vol_sold                  0
dtype: int64

### Every city that has a zip code missing is also missing an address and city. The store number and name are the only columns that can be used to identify a particular store in these cases.

## 3.1 zip_code column: drop non-numeric characters

In [34]:
df.zip_code.isna().sum()

2420

In [35]:
df.zip_code.shape[0], df.zip_code.str.isnumeric().sum()

(12591077, 10083835)

In [36]:
print(df.zip_code.shape[0] - df.zip_code.str.isnumeric().sum(), 'zip code rows without total numerics.')

2507242 zip code rows without total numerics.


In [37]:
import re
def remove_non_nums(i_str):
    return re.sub(r'\D', '', str(i_str))

In [38]:
df.zip_code = df.zip_code.apply(remove_non_nums)

In [39]:
df.zip_code.shape[0], df.zip_code.str.isnumeric().sum()

(12591077, 12588657)

In [40]:
print(df.zip_code.shape[0] - df.zip_code.str.isnumeric().sum(), 'zip code rows without total numerics.')

2420 zip code rows without total numerics.


In [41]:
df.zip_code.isna().sum()

0

In [42]:
df.zip_code.head()

0    52002
1    52001
2    52040
3    51351
4    52627
Name: zip_code, dtype: object

## 3.2. Address and City

In [43]:
df.query('address.isna() & city.notna()', engine='python')

,date,year,month,day,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_vol,bottle_cost,state_bottle_retail,bottles_sold,sale,vol_sold
11755302,2017-06-22,2017,6,22,9932,"3-Oaks Distillery, LLC",NaN,Dubuque,520020,"Dubuque 52002\n(42.516027, -90.766335)",31,DUBUQUE,1011100.0,Blended Whiskies,215.0,"3-Oaks Distillery, LLC",27890,3-Oaks Distillery Whiskey,12,750,$21.60,$32.40,1,$388.80,0.75


### The address column has only one NaN value more than the city column, making the address of the store easy to ascertain via search engine: index: 11757989; address: 23039 Pfeiler Rd, Holy Cross, IA 52053

In [44]:
df.address[11757989] = '23039 Pfeiler Rd'

<ipython-input-44-bdedd23ba3a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.address[11757989] = '23039 Pfeiler Rd'


## 4. Vendor_number

### vendor_number does not refer to only one name, as shown in the query below.

In [45]:
df.query('vendor_number == 255.0').vendor_name.value_counts()

Wilson Daniels Ltd.    68802
Infinium Spirits       10235
WILSON DANIELS LTD      9722
Name: vendor_name, dtype: int64

### As an aside, notice that the item description of some rows match, yet the item numbers differ, as well as the category_name and category_number. What compounds the fact that some differ despite the fact that they were purchased on the same day at the same store (store_name == 'Central City Liquor, Inc.', store_number == 2190). This indicates a surprising discrepancy between such columns.

# Data Cleaning:
## Remove non-numerics from columns dealing with money: bottle_cost, state_bottle_retail, and sale

In [46]:
df.sale = df.sale.replace(to_replace='nan', value=0)
df.bottle_cost = df.bottle_cost.replace(to_replace='nan', value=0)

In [47]:
# Change dtypes of columns to iterables
df.bottle_cost = df.bottle_cost.astype(str)
df.state_bottle_retail = df.state_bottle_retail.astype(str)
df.sale = df.sale.astype(str)


# Convert nan strings to a string of the number 0.
df.bottle_cost = df.bottle_cost.replace(to_replace='nan', value='0')
df.query('bottle_cost == "nan"').bottle_cost

df.state_bottle_retail = df.state_bottle_retail.replace(to_replace='nan', value='0')
df.query('state_bottle_retail == "nan"').state_bottle_retail

df.sale = df.sale.replace(to_replace='nan', value='0')
df.query('sale == "nan"').sale


# Strip "$"
df.bottle_cost = [x.strip('$') for x in df.bottle_cost]
df.state_bottle_retail = [x.strip('$') for x in df.state_bottle_retail]
df.sale = [x.strip('$') for x in df.sale]


# Convert back to float
df.bottle_cost = pd.to_numeric(df.bottle_cost, downcast='float')
df.state_bottle_retail = pd.to_numeric(df.state_bottle_retail, downcast='float')
df.sale = pd.to_numeric(df.sale, downcast='float', errors='ignore')

In [48]:
df.bottle_cost.dtype, df.state_bottle_retail.dtype, df.sale.dtype

(dtype('float32'), dtype('float32'), dtype('float32'))

In [49]:
print(df.query('state_bottle_retail == 0 | bottle_cost == 0 | sale == 0').shape[0],
'rows of the above contain zero. They were NaN before converted to 0.',
'\n', df.query('bottle_cost == 0 & sale == 0').shape[0], 'rows contain NaNs for all three columns: state_bottle_retail, bottle_cost, sale.',
'\n\t-', df.query('state_bottle_retail == 0').shape[0], 'rows are state_bottle_retail.',
'\n\t-', df.query('bottle_cost == 0').shape[0], 'rows are bottle_cost.',
'\n\t-', df.query('state_bottle_retail == 0 & bottle_cost == 0').shape[0], 'rows are both state_bottle_retail and bottle_cost.'
'\n', df.query('sale == 0').shape[0], 'rows are sale alone, with', 4316 - 3094, 'more than the other factors.')

4316 rows of the above contain zero. They were NaN before converted to 0. 
 3094 rows contain NaNs for all three columns: state_bottle_retail, bottle_cost, sale. 
	- 3094 rows are state_bottle_retail. 
	- 3094 rows are bottle_cost. 
	- 3094 rows are both state_bottle_retail and bottle_cost.
 4316 rows are sale alone, with 1222 more than the other factors.


### Why are there more missing values for the sales column?

### The following two cells are a significant find that help explain why sale information was missing when the accompanying columns (like bottle_cost) were filled. 
#### - The first variable is high (61%)
#### - The percentages differ greatly from the dataset
#### - The values are ordered meaningfully that can tell a story: the sale price was not well recorded when the bottles sold were few, especially when it was 1-3 bottles, and much especially when it was only 1 bottle. <br>
### In terms of practical business, this should be communicated to business owners interested in collecting data in order to optimize the amount of data recorded.

In [50]:
df.query('sale == 0 & bottle_cost != 0').bottles_sold.value_counts(normalize=True)

1     0.610475
2     0.153028
3     0.103928
4     0.048282
6     0.022913
5     0.018822
11    0.013912
9     0.008183
12    0.005728
10    0.004910
8     0.004092
21    0.001637
25    0.000818
7     0.000818
24    0.000818
23    0.000818
0     0.000818
Name: bottles_sold, dtype: float64

In [51]:
df.bottles_sold.value_counts(normalize=True)

12     2.149120e-01
1      2.051150e-01
6      1.632906e-01
2      1.489820e-01
3      1.080358e-01
           ...     
175    7.942132e-08
996    7.942132e-08
173    7.942132e-08
171    7.942132e-08
154    7.942132e-08
Name: bottles_sold, Length: 450, dtype: float64

In [52]:
print(round(df.query('sale == 0 & bottles_sold == 1').shape[0] / df.query('sale == 0').shape[0] * 100, 1),
'percent of NaN values in the sale column happened when only one bottle was sold.')

25.8 percent of NaN values in the sale column happened when only one bottle was sold.


# Test category column and save dataset

In [53]:
df.category.head(3)

0    1062200.0
1    1081900.0
2    1081335.0
Name: category, dtype: float64

In [54]:
df.to_csv('/Users/charisameeker/Documents/Data/iowa_liquor_clean1.5.csv')

In [58]:
df.category.dtype

dtype('float64')

In [55]:
stop

NameError: name 'stop' is not defined

# Data Cleaning and EDA:
## Cleaning What We Have

In [59]:
# make all strings lowercase
object_column_list = list(df.dtypes[df.dtypes == object].index)
# object_column_list.remove('store_location')
# We will need Store Location later:
for object_column in object_column_list:
    df.loc[:,object_column] = df.loc[:,object_column].str.lower().str.strip().str.split().str.join(' ')
    print(object_column)
    gc.collect()

store_name
address
city
zip_code
store_location
county
category_name
vendor_name
item_description


In [60]:
from tqdm.notebook import trange, tqdm

# Check if any columns can be converted to categories:
count_dict = {}
for c in tqdm(df.columns):
    count_per_value = df[c].value_counts()
    count_dict[c] = {
        'count_of_values': count_per_value.count(),
        'count_per_value': count_per_value
    }
    print(f"Column {c} has {count_dict[c]['count_of_values']} values.")

Column date has 1379 values.
Column year has 6 values.
Column month has 12 values.
Column day has 31 values.
Column store_number has 1884 values.
Column store_name has 1934 values.
Column address has 1846 values.
Column city has 416 values.
Column zip_code has 881 values.
Column store_location has 3083 values.
Column county_number has 100 values.
Column county has 103 values.
Column category has 108 values.
Column category_name has 108 values.
Column vendor_number has 271 values.
Column vendor_name has 357 values.
Column item_number has 7395 values.
Column item_description has 5811 values.
Column pack has 30 values.
Column bottle_vol has 49 values.
Column bottle_cost has 2768 values.
Column state_bottle_retail has 3224 values.
Column bottles_sold has 450 values.
Column sale has 23154 values.
Column vol_sold has 1180 values.



In [61]:
df.category.head(3)

0    1062200.0
1    1081900.0
2    1081335.0
Name: category, dtype: float64

### Interesting observations:

The most of object columns should be converted to categories as even Item_Description column has only 4k+ values for 12M rows.

- store_location has address, zip, city AND coordinates!
- store_number.count < store_name.count < store_address - either we have typos in names/addresses or some stores - change location over time
- county_number.count < county.count - we have TYPOS or NAME CONVENTION CHANGES in county names
- vendor_number.count < vednor_name.count - same as above
- categoy != category_name
- item_number != item_name -> could be that name is preserved while some other attribute of the item is changed
- volume Sold in Liters has less values than in Gallons <br><br>

So we need some basic cleaning before we cast objects to categories.

In [62]:
df.to_csv('/Users/charisameeker/Documents/Data/iowa_liquor_clean1.6.csv')

## Store Location

In [63]:
# Note: we are not using count_dict as we will use the function for batch processing new data:

def get_coordinates(location_series):
    
    # Get unique values:
    unique_locations = location_series.unique()
    
    # Create Pandas series in order to use .str methods:
    unique_loactions = pd.Series(unique_locations, index=unique_locations)
    
    # Split every series value using '\n'
    # Take the last element from the list 
    # Convert series to a dictionary
    # (Note: [-1:] trick this helps to dodge empty lists)
    location_dict = unique_loactions.str.split('\n').str[-1:].str[0].to_dict()

    # Use the dict to map Locations to Coordinates:
    return location_series.map(location_dict, na_action='ignore')

# Create a new column coordinates
df.loc[:,'coordinates'] = get_coordinates(df['store_location'])

# Get rid of useless column:
# df.drop(columns=['store_location'], inplace=True)
gc.collect()

# Check 5 rows of new column:
df.coordinates[:5]

0    4400 asbury rd dubuque 52002 (42.515282, -90.7...
1    400 locust st dubuque 52001 (42.497219, -90.66...
2    733 16th ave se dyersville 52040 (42.470204, -...
3    1610 okoboji avenue milford 51351 (43.331525, ...
4                    11802 avenue h fort madison 52627
Name: coordinates, dtype: object

In [64]:
df.category.head(3)

0    1062200.0
1    1081900.0
2    1081335.0
Name: category, dtype: float64

In [65]:
df.to_csv('/Users/charisameeker/Documents/Data/iowa_liquor_clean1.7.csv')

## Stores

In [66]:
# Stores:
stores_df = df.loc[:,['store_number','store_name','address','city']].drop_duplicates()

# Count values for every store number:
store_number_counts = stores_df.loc[:,"store_number"].value_counts()

# Find store counts with multiple name or address antries:
idx_store_duplicates = store_number_counts[store_number_counts!=1].index

In [67]:
# An example of multiple records:
stores_df.loc[stores_df["store_number"]==idx_store_duplicates[5]].drop_duplicates()

,store_number,store_name,address,city
1335489,4783,casey's general store #1684 / emmets,"2406, main st",emmetsburg
9954098,4783,casey's general store #1684 / emmetsburg,"2406, main st",emmetsburg
10527582,4783,casey's general store #1684 / emmetsburg,NaN,NaN
10636315,4783,casey's general store #1684 / emmetsburg,2406 main st,emmetsburg


In [68]:
stores_df.shape

(2141, 4)

In [69]:
df.category.head(3)

0    1062200.0
1    1081900.0
2    1081335.0
Name: category, dtype: float64

Feautures:<br>

- Store chains share name "Hy-Vee Wine and Spirits", but have different subnames: "Estherville". The fact that a store is part if a chain could be a valuable feature.

- Note that some stores have format, so names won't match due to the number: cvs pharmacy #4816 / council bluffs

Fixes:

Apparently we have both typos, changes in conventions and (the worst) shop relocations. Possible fix:

- split by '/', strip - to split subname from name ie 'Hy-Vee Wine and Spirits / Estherville' == 'Hy-Vee Wine and Spirits', 'Estherville'
- StoreName, SubName is the name of max length for given number and address
- Address NaN fill forward with the last known address
- Coordinates are inconsistent and probably unreliable,so we will have to create new geotags.

In [80]:
df.dtypes

date                   datetime64[ns]
year                            int16
month                            int8
day                              int8
store_number                    int16
store_name                     object
address                        object
city                           object
zip_code                       object
store_location                 object
county_number                    int8
county                         object
category                      float64
category_name                  object
vendor_number                 float64
vendor_name                    object
item_number                     int32
item_description               object
pack                            int16
bottle_vol                      int64
bottle_cost                   float32
state_bottle_retail           float32
bottles_sold                    int16
sale                          float32
vol_sold                      float32
coordinates                    object
store_subnam

In [70]:
df.to_csv('/Users/charisameeker/Documents/Data/iowa_liquor_clean1.8.csv')

In [71]:
# Process Store Name:
df[['store_name','store_subname']] = df['store_name'].str.rsplit(pat=" / ", expand=True, n=1)
df[['store_name','store_subnumber']] = df['store_name'].str.rsplit(pat=" #", expand=True, n=1)

In [72]:
# Lets do the same trick for vendors:
def max_length(df, number_column, name_column):
    # Stores:
    stores_df = df.loc[:,[number_column, name_column]].drop_duplicates()
    
    # Create dictionary to map CountyNumber to max CountyName
    max_dict = stores_df.fillna('#').groupby(number_column)[name_column].max().to_dict()
    
    # Replace CountyNames with max CountyNames using the dictionary:
    return df[number_column].map(max_dict)

In [73]:
df.loc[:,'store_name'] = max_length(df,'store_number','store_name')
df.loc[:,'stores_subname'] = max_length(df,'store_number','store_subname')

In [74]:
l = df['store_name'].str.extract(pat=r'(\d+$)')

In [81]:
df.category.head(3), max(df.category)

(0    1062200.0
 1    1081900.0
 2    1081335.0
 Name: category, dtype: float64,
 1901200.0)

In [78]:
df.to_csv('/Users/charisameeker/Documents/Data/iowa_liquor_clean1.9.csv')

# After csv

In [82]:
df.category.isna().sum()

0

## County Names and Numbers
### county has 99 values, county_name has 103 values.

In [ ]:
df.loc[:,'county'] = max_length(df,'county_number', 'county')

## Vendors Names & Numbers
### Column Vendor Number has 202 values. Column Vendor Name has 271 values.

In [ ]:
df.loc[:,'vendor_name'] = max_length(df,'vendor_number', 'vendor_name')

## Category Names & Numbers
### Column Category has 102 values. Column Category Name has 106 values.

In [ ]:
# Lets do the sam trick for Categories:
df.loc[:,'category'] = max_length(df,'category','category_name')

In [ ]:
df.category.head(), df.category_name.head()

# Save data: iowa_liquor_clean1.4.pkl

In [1]:
df.to_pickle('/Users/charisameeker/Documents/Data/iowa_liquor_clean1.5.pkl')
# df.to_csv('/Users/charisameeker/Documents/Data/iowa_liquor_clean1.5.csv')

NameError: name 'df' is not defined